In [81]:
# This is for the COMP9417 Group Project
# The problem is the OTTO in the Kaggle
# Source: https://www.kaggle.com/c/otto-group-product-classification-challenge/overview
# RandomForest / conception / svm / knn / Naive Bayesian

In [82]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans

In [83]:
raw_train_data = pd.read_csv("D:\\train.csv")
raw_test_data = pd.read_csv("D:\\test.csv")

In [84]:
print(raw_train_data.shape)
print(raw_test_data.shape)

(61878, 95)
(144368, 94)


In [85]:
Train_data = raw_train_data.iloc[:, 1:95]
labels = set(Train_data.iloc[:,93:94].target)
Valid_x = np.array(raw_test_data.iloc[:, 1:94])
Total_x = np.array(Train_data.iloc[:,0:93])
Total_y = np.array(Train_data.iloc[:,93:94])

In [86]:
print(f"The current has the following label {labels}")
print(f"The shape the Total_X {Total_x.shape}")
print(f"The shape the Total_Y {Total_y.shape}")
print(f"The shape the Valid_X {Valid_x.shape}")
print(f"Check Nan in Total_X {set(np.isnan(Total_x).any(axis=1))}")
# print(f"Check Nan in Train_Y {np.isnan(Train_Y).any(axis=1)}")
print(f"Check Nan in Valid_X {set(np.isnan(Valid_x).any(axis=1))}")

The current has the following label {'Class_2', 'Class_5', 'Class_9', 'Class_4', 'Class_7', 'Class_3', 'Class_1', 'Class_8', 'Class_6'}
The shape the Total_X (61878, 93)
The shape the Total_Y (61878, 1)
The shape the Valid_X (144368, 93)
Check Nan in Total_X {False}
Check Nan in Valid_X {False}


In [87]:
Total_X,Total_Y = shuffle(Total_x,Total_y)

In [88]:
# In this part using the previous 50000 to do the training and the last data to do the test check the score
Train_X = Total_X[:50000]
Train_Y = Total_Y[:50000]
Test_X = Total_X[50000:]
Test_Y = Total_Y[50000:]
print(Train_X.shape)
print(Train_Y.shape)
print(Test_X.shape)
print(Test_Y.shape)

(50000, 93)
(50000, 1)
(11878, 93)
(11878, 1)


In [89]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

In [90]:
# The n_neighbors=3 is due to KNeighboursClassifier [k=3]
knn_classifier = KNeighborsClassifier(n_neighbors=9)
# fit the knn classifier
knn_classifier.fit(Train_X ,Train_Y.ravel())

KNeighborsClassifier(n_neighbors=9)

In [91]:
predict_y = knn_classifier.predict(Test_X)

In [92]:
normal_KNN_report = metrics.classification_report(Test_Y,predict_y)

In [93]:
print(normal_KNN_report)

              precision    recall  f1-score   support

     Class_1       0.65      0.53      0.59       391
     Class_2       0.68      0.85      0.76      3109
     Class_3       0.55      0.47      0.51      1539
     Class_4       0.69      0.24      0.35       551
     Class_5       0.93      0.97      0.95       524
     Class_6       0.94      0.93      0.94      2654
     Class_7       0.75      0.58      0.65       523
     Class_8       0.90      0.90      0.90      1601
     Class_9       0.85      0.87      0.86       986

    accuracy                           0.78     11878
   macro avg       0.77      0.70      0.72     11878
weighted avg       0.78      0.78      0.77     11878



In [94]:
# Best C: 0.1
# Best Score: 0.75044
best_C = 2

In [103]:
svc = LinearSVC(C=best_C, random_state=0)
svc.fit(Train_X, Train_Y)
svm_pred = svc.predict(Test_X)

E:\Anaconda\envs\tf24\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
E:\Anaconda\envs\tf24\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [104]:
normal_svm_predict = metrics.classification_report(Test_Y,svm_pred)
print(normal_svm_predict)

              precision    recall  f1-score   support

     Class_1       0.82      0.17      0.29       391
     Class_2       0.64      0.77      0.70      3109
     Class_3       0.46      0.45      0.45      1539
     Class_4       0.83      0.10      0.18       551
     Class_5       0.97      0.96      0.96       524
     Class_6       0.92      0.93      0.93      2654
     Class_7       0.63      0.62      0.63       523
     Class_8       0.86      0.89      0.88      1601
     Class_9       0.76      0.89      0.82       986

    accuracy                           0.74     11878
   macro avg       0.77      0.64      0.65     11878
weighted avg       0.75      0.74      0.72     11878



In [97]:
# XGboost

In [98]:
import xgboost as xgb

In [99]:
xgb_train_X = xgb.DMatrix(Train_X)
pre_num_y = [[int(i[-1][-1])] for i in Train_Y]
# print(pre_num_y)
xgb_train_Y = xgb.DMatrix(np.array(pre_num_y))
Train_Total = xgb.DMatrix(Train_X,pre_num_y)

In [100]:
xgb_classifier = xgb.XGBClassifier(n_estimators=93,\
                                   max_depth=4, \
                                   learning_rate=0.1, \
                                   subsample=0.7, \
                                   colsample_bytree=0.7)

In [101]:
X_test = xgb.DMatrix(Test_X)

In [102]:
params = {"objective": "multi:softprob", "eval_metric":"mlogloss", "num_class": 9}
xgb_classifier.train(params,Train_Total,180)

AttributeError: 'XGBClassifier' object has no attribute 'train'

In [ ]:
preds_xgb = xgb_classifier.predict(Test_X)

In [ ]:
normal_xgb_predict = metrics.classification_report(Test_Y,preds_xgb)
print(normal_svm_predict)

              precision    recall  f1-score   support

     Class_1       0.66      0.25      0.36       387
     Class_2       0.60      0.92      0.73      3084
     Class_3       0.53      0.24      0.33      1546
     Class_4       0.89      0.11      0.20       552
     Class_5       0.96      0.94      0.95       521
     Class_6       0.91      0.92      0.92      2646
     Class_7       0.75      0.52      0.61       582
     Class_8       0.86      0.90      0.88      1654
     Class_9       0.81      0.85      0.83       906

    accuracy                           0.74     11878
   macro avg       0.78      0.63      0.64     11878
weighted avg       0.75      0.74      0.71     11878



In [ ]:
kmeans = KMeans(n_clusters=9, random_state=0).fit(Train_X,Train_Y)

In [ ]:
kmns_pred = kmeans.predict(Test_X)

In [ ]:
result_pred_kmns = [[f"Class_{i}"] for i in kmns_pred]
# print(result_pred_kmns)

In [ ]:
KMNS_REPORT = metrics.classification_report(Test_Y,result_pred_kmns)
print(KMNS_REPORT)

              precision    recall  f1-score   support

     Class_0       0.00      0.00      0.00         0
     Class_1       0.00      0.00      0.00       387
     Class_2       0.00      0.00      0.00      3084
     Class_3       0.00      0.00      0.00      1546
     Class_4       0.00      0.00      0.00       552
     Class_5       0.01      0.01      0.01       521
     Class_6       0.00      0.00      0.00      2646
     Class_7       0.00      0.00      0.00       582
     Class_8       0.21      0.10      0.14      1654
     Class_9       0.00      0.00      0.00       906

    accuracy                           0.01     11878
   macro avg       0.02      0.01      0.01     11878
weighted avg       0.03      0.01      0.02     11878



E:\Anaconda\envs\tf24\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Anaconda\envs\tf24\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Anaconda\envs\tf24\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Anaconda\envs\tf24\lib\site-packages\sklearn\metrics\_classification.